- load submission with iterative imp and reconstruct dataset with it - DONE

- for col in cols with missing values (or only F2_x and F4_x ?)
    - col is target
    - other cols are features
    - complete rows are for training
    - incomplete (with missing value) are for prediction
    - when the prediction is complete, update column values in dataset

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from tqdm import tqdm

# Load Data

In [2]:
data = pd.read_csv('../input/tabular-playground-series-jun-2022/data.csv', index_col = 'row_id')
submission = pd.read_csv('../input/tabular-playground-series-jun-2022/sample_submission.csv', index_col = 'row-col')
sub_iter = pd.read_csv('../input/submission-iterativeimp/submission_iterative-imp.csv', index_col = 'row-col')

# display(data,submission,sub_iter)

# Apply results from Iterative Imputer

In [3]:
# use results from iterative imputation as a base for next steps -> fill values into data
data_imp = data.copy()
for i in tqdm(sub_iter.index):
    row = int(i.split('-')[0])
    col = i.split('-')[1]
    data_imp.at[row,col] = sub_iter.loc[i]

100%|██████████| 1000000/1000000 [01:57<00:00, 8517.90it/s]


# Treat cols with missing values as targets

In [4]:
%%time
 
from xgboost import XGBRegressor

model = XGBRegressor()

cols = data.columns[data.isnull().any()]
# start just with colums F_4_X
cols = [c for c in cols if c.startswith('F_4_')]
display(cols)
#display(data_imp['F_4_5'])

for col in cols:
    # prepare dframes
    train_df = data_imp[data[col].notna()]
    test_df = data_imp[data[col].isna()]
    
    X_train = train_df.copy()
    y_train = X_train.pop(col)
    X_test = test_df.drop(col,axis=1)
    
#     display(X_train.columns, X_test.columns)
    
#     display(train_df,test_df)
#     display(X_train,y_train,X_test)
#     display(X_train,y_train,X_test)
    
    # model: train + predict
    model.fit(X_train,y_train)
    y = model.predict(X_test)
    
    # update df with new predictions
    data_imp.loc[data[col].isna(),col] = y
    
# display(data_imp['F_4_5'])

['F_4_0',
 'F_4_1',
 'F_4_2',
 'F_4_3',
 'F_4_4',
 'F_4_5',
 'F_4_6',
 'F_4_7',
 'F_4_8',
 'F_4_9',
 'F_4_10',
 'F_4_11',
 'F_4_12',
 'F_4_13',
 'F_4_14']

CPU times: user 12h 39s, sys: 56.2 s, total: 12h 1min 35s
Wall time: 3h 6min 23s


# Submission

In [5]:
for i in tqdm(submission.index):
    row = int(i.split('-')[0])
    col = i.split('-')[1]
    submission.loc[i,'value'] = data_imp.loc[row,col]
    
    
submission.to_csv('submission.csv')

100%|██████████| 1000000/1000000 [01:43<00:00, 9702.81it/s]
